## CASE 2: Non-Inverting with Negative Offset, b is negative so applied to V-
<div>
<img src="images/case2.png" width="500"/>
</div>
(We can add $V_{inv}$ as it is negative) 

Having $V_{ref}$ shortened to $GND$ we have a _Noninverting Amplifier_

$$
V_{outn}=V_{in}\dfrac{(R_f+R_g)}{R_g}
$$

whith $m$

$$
m=\dfrac{(R_f+R_g)}{R_g}
$$

Having $V_{in}$ shortened to $GND$ we have a _Inverting Amplifier_

$$
V_{outi} = V_{ref} \dfrac{-R_f} {R_g}
$$

with $b$

$$
b=V_{ref}\dfrac{-R_f}{R_g}
$$


The sum is $V_{out}$

$$
V_{out}=V_{outn}+V_{outi}
$$

We assume that $V_{ref}$ is the voltage needed to provide the given output. To generate $V_{ref}$ from $V_{cc}$ or other voltage source see below. 

<!-- With $V_{ninv}$ having $V_{ref}$ shortened to ground $GND$ so that we have a Noninverting Amplifier which represents $m$

$$
V_{ninv}=V_{in}\dfrac{(R_f+R_g+R_{th})}{(R_g+R_{th})}
$$

With $V_{ninv}$ having $V_{in}$ shortened to ground $GND$ so that we have a Inverting Amplifier which represents |$b$|

$$
V_{inv}=V_{th}\dfrac{-R_f}{(R_g+R_{th})}
$$

With $R_{th}$ representing the Thevenin equivalent of $R_1$ and $R_2$

$$
R_{th}=R_1\parallel R_2
$$

With $V_{th}$ representing the Thevenin equivalent of $V_{R2}$

$$
V_{th}=\dfrac{R_2*V_{ref}}{R_1+R_2}
$$ -->

Assumptions:
Noninverting OpAmp
Gain: A=Vout/Vin
V+=V- always

With V+=V- we have V-=Vin as Vin is at V+ pin

<!--
At the resistor divider we have Rg/(Rg+Rf)=Vin/Vout which is Vout/Vin=(Rg+Rf)/Rg which is Vout/Vin=1+Rf/Rg => That is Vout=Vin*(1+Rf/Rg) which is m.
Now we have to subtract b
Lets ground V+, we have a inverting Amp now!
Vout/Vref=-Rf/Rg

So we add all the Outputs together Vout=Vout(Noninverting)+Vout(Inverting)


https://www.circuitlab.com/editor/#?id=n665q67teq8n

Rth = R4 ∥ R6 = R4×R6 ÷ (R4+R6)
Vth = V1 × R6 ÷ (R4+R6) -->




In [16]:
from ESeries import ESeries
#Define the E_Series to chose the Resistors from 
series_range= ESeries.Series.E24

In [17]:
%matplotlib inline
#%pip install sympy numpy pyspice dill
import dill
#dill.load_session(session_file_name)
import sympy as sp
from sympy import solve, symbols, Eq, S
from IPython.display import Math, display

# Importing packages
import matplotlib.pyplot as plt
import numpy as np

from operator import itemgetter
from IPython.display import Markdown as md

from ESeries import ESeries

sp.init_printing()
series= ESeries()


In [18]:
%store -r m
%store -r b
%store -r x1
%store -r y1
%store -r x2
%store -r y2
%store -r Vx
m=m
b=b
x1=x1
y1=y1
x2=x2
y2=y2

#chosen Vx, the voltage to generate Vref
vx_chosen=Vx
m,b,x1,y1,x2,y2


In [19]:
rf,rg, vref= symbols('R_f R_g V_ref')

Solve for $R_g$ for given slope $m$
$$
m=\dfrac{R_f+R_g}{R_g}
$$

In [20]:
#we use inverted m and normal voltage divider rule
voltage_divider_ratio=series.voltage_divider(ratio=(float(1/m)),series=series_range, r1_min=80e3)
rf_series_value=voltage_divider_ratio['r1']
rg_series_value=voltage_divider_ratio['r2']
delta_ratio=voltage_divider_ratio['ratio_difference']
(rg_series_value,rf_series_value,delta_ratio)

[{'r1': 82000.0000000000, 'r2': 91000.0000000000, 'ratio_difference': 0.733988439306358}, {'r1': 82000.0000000000, 'r2': 82000.0000000000, 'ratio_difference': 0.760000000000000}, {'r1': 91000.0000000000, 'r2': 91000.0000000000, 'ratio_difference': 0.760000000000000}, {'r1': 82000.0000000000, 'r2': 75000.0000000000, 'ratio_difference': 0.782292993630573}, {'r1': 100000.000000000, 'r2': 91000.0000000000, 'ratio_difference': 0.783560209424084}]


In [21]:
#Solve for Rg in respect to m
rg_eq=Eq(m,(rf+rg)/rg)
rg=solve(rg_eq,[rg])[0]
rg

Supply a chosen value for $R_f$ to get $R_g$

In [22]:
rg.subs(rf,rf_series_value)


In [23]:
md(f"### E-Series {series_range.value} Resistor value for $R_g = {rg_series_value:.1f} \Omega$")

### E-Series 24 Resistor value for $R_g = 91000.0 \Omega$

Solve for $V_{ref}$ for a given intercept $b$
$$
b=V_{ref}* \dfrac{-R_f}{R_g}
$$

In [24]:
vref=solve(Eq(b, vref*(-rf/rg)),[vref])[0]
vref=vref.subs(rf,rf_series_value)
vref

In [25]:
md(f"### $R_f = {rf_series_value:.0f}\Omega , R_g = {rg_series_value:.0f}\Omega , V_ref= {float(vref):.3}V$")

### $R_f = 82000\Omega , R_g = 91000\Omega , V_ref= -21.9V$

In [26]:
#verification
vout1=x1*(rf+rg)/rg+vref*-rf/rg
vout1=vout1.subs(rf,rf_series_value)
(x1,vout1)

In [27]:
#verification
vout2=x2*(rf+rg)/rg+vref*-rf/rg
vout2=vout2.subs(rf,rf_series_value)
(x2,vout2)

As we have $V_{ref}$ applied to the negative input of the opamp the voltage divider will be loaded through $I_{rg}$ and $I_{rf}$. To reduce the Error, we use a buffered voltage divider.

<div>
<img src="images/buffered-voltage-divider.png" width="500"/>
</div>

In [28]:
md(f"## Proceed to [Voltage Divider](./03-voltage-divider.ipynb)")

## Proceed to [Voltage Divider](./03-voltage-divider.ipynb)

In [29]:
rg=rg_series_value
rf=rf_series_value

In [30]:
# Keep this vales as we need them in the Notebooks for the different cases
%store rf
%store vx_chosen
%store vref
%store rg



Stored 'rf' (Float)
Stored 'vx_chosen' (int)
Stored 'vref' (Rational)
Stored 'rg' (Float)
